In [1]:
from dcat_ap_hub import Dataset

# url = "http://localhost:8081/datasets/ba578d9a-2f7d-42b7-b7a8-a404a64979ec.jsonld"
# url = "http://localhost:8081/datasets/96d4eca1-4e85-4e46-895d-b84577b31398.jsonld"
# url = "http://localhost:8081/datasets/188aad26-d043-4189-81b9-583ed5d489c0.jsonld"
url = "http://localhost:8081/datasets/3dd505c9-b146-4efb-a2dd-d4c5c116882e.jsonld"
url = "http://localhost:8081/datasets/6a207485-a38f-4213-b6dc-767dfd643525.jsonld?useNormalizedId=true&locale=en"


ds = Dataset.from_url(url)
raws = ds.download(data_dir="./data")
files = ds.process(processed_dir="./processed")

2026-02-27 14:41:15,763 - dcat-ap-hub - INFO - Dataset directory exists: data/IoT Integrated Predictive Maintenance Dataset. Skipping download.


Processed data found at 'processed'. Skipping (use force=True to rerun).


In [2]:
list(raws)

[<File: predictive_maintenance_dataset.csv (💤 Lazy)>,
 <File: dcat-metadata.jsonld (💤 Lazy)>,
 <File: notebook_pmd.ipynb (💤 Lazy)>,
 <File: processor_pmd.py (💤 Lazy)>,
 <File: processor_pmd.cpython-311.pyc (💤 Lazy)>,
 <File: y_train.npy (💤 Lazy)>,
 <File: column_transformer.pkl (💤 Lazy)>,
 <File: y_test.npy (💤 Lazy)>,
 <File: X_test.npy (💤 Lazy)>,
 <File: X_train.npy (💤 Lazy)>]

In [3]:
raws["predictive_maintenance_dataset.csv"].data.head()

timestamp machine_id  vibration  acoustic  temperature  current  \
0  2024-07-01 08:00:00        M01      0.822     0.645        66.85    13.04   
1  2024-07-01 08:01:00        M01      1.398     0.834        76.20    15.08   
2  2024-07-01 08:02:00        M01      0.856     0.590        67.03    12.30   
3  2024-07-01 08:03:00        M01      0.793     0.544        65.04    11.69   
4  2024-07-01 08:04:00        M01      1.279     0.721        78.19    14.84   

   IMF_1  IMF_2  IMF_3  label  
0  0.196  0.033  0.000      0  
1  0.345  0.132  0.001      1  
2  0.187  0.017  0.002      0  
3  0.196 -0.060  0.003      0  
4  0.330 -0.115  0.004      1

In [4]:
print(ds)

📊 Dataset('IoT Integrated Predictive Maintenance Dataset', 1 distros) [Data: IoT Integrated Predictive Maintenance Dataset, Processed: ✓]


In [5]:
X_train = files["X_train.npy"].data
y_train = files["y_train.npy"].data
X_test = files["X_test.npy"].data
y_test = files["y_test.npy"].data

print(f"Train shape: X={X_train.shape}, y={y_train.shape}")
print(f"Test shape: X={X_test.shape}, y={y_test.shape}")

Train shape: X=(1440, 12), y=(1440,)
Test shape: X=(360, 12), y=(360,)


In [8]:
from tabpfn.constants import ModelVersion
from sklearn.metrics import accuracy_score
from tabpfn import TabPFNClassifier

clf = TabPFNClassifier.create_default_for_version(ModelVersion.V2)

clf.fit(X_train[:2], y_train[:2])

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

Accuracy 0.8861111111111111


In [6]:
from tabpfn.constants import ModelVersion
from sklearn.metrics import mean_squared_error, r2_score
from tabpfn import TabPFNRegressor

clf = TabPFNRegressor.create_default_for_version(ModelVersion.V2)

clf.fit(X_train[:1000], y_train[:1000])

# Predict labels
predictions = clf.predict(X_test[:1000])
mse = mean_squared_error(y_test[:1000], predictions)
r2 = r2_score(y_test[:1000], predictions)

print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)

Mean Squared Error (MSE): 116.83656685685769
R² Score: -1337.214534656382
